<h1 align="center">Changes in sentinel measures of primary care activity during the pandemic</h1>

The purpose of this notebook is to provide measures of overall activity at the practice level during the pandemic.

The following sentinel measures are provided:
* [O/E - Systolic BP reading](#systolic_bp)
* [QRISK2 cardiovascular disease 10 risk score](#qrisk2)
* [Serum Cholesterol level](#cholesterol)
* [ALT level](#ALT)
* [Serum TSH level](#serum_tsh)
* [Red blood cell count](#rbc_fbc)
* [Haemoglobin A1c level - IFCC standardised](#hba1c)
* [Serum Sodium level](#serum_sodium)
* [Asthma annual review](#asthma)
* [Chronic obstrutive pulmonary disease annual review](#copd)

In [10]:
from IPython.display import HTML
import pandas as pd
import matplotlib.pyplot as plt
from utilities import *

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [11]:
sentinel_measures = ["qrisk2", "asthma", "copd", "sodium", "cholesterol", "alt", "tsh", "alt", "rbc", 'hba1c', 'systolic_bp']

data_dict = {}

for measure in sentinel_measures:
    df = load_and_drop(measure)
    data_dict[measure] = df
    
data_dict_practice = {}

for measure in sentinel_measures:
    df = load_and_drop(measure, practice=True)
    data_dict_practice[measure] = df

sentinel_measure_codelist_mapping_dict = {"systolic_bp":"user-richard-croker-systolic-blood-pressure-qof", "qrisk2":"user-richard-croker-cvd-risk-assessment-score-qof", "cholesterol": "user-richard-croker-cholesterol-tests-numerical-value", "alt": "user-richard-croker-alanine-aminotransferase-alt-tests-numerical-value", "tsh": "user-richard-croker-thyroid-stimulating-hormone-tsh-tests-numerical-value", "rbc": "user-richard-croker-red-blood-cell-rbc-tests-numerical-value", "hba1c": "user-richard-croker-glycated-haemoglobin-hba1c-tests", "sodium": "user-richard-croker-sodium-tests-numerical-value", "asthma": "user-richard-croker-asthma-annual-review-qof", "copd": "user-richard-croker-chronic-obstructive-pulmonary-disease-copd-review-qof"}

codelist_dict = {}
for measure in sentinel_measures:
    codelist_name = sentinel_measure_codelist_mapping_dict[measure]
    codelist = pd.read_csv(f'../codelists/{codelist_name}.csv')
    codelist_dict[measure] = codelist

FileNotFoundError: [Errno 2] No such file or directory: '../output/measure_qrisk2.csv'

<a id="systolic_bp"></a>
### O/E - Systolic BP

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'systolic_bp', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="qrisk"></a>
### QRISK2 Cardiovascular Disease 10 year risk score

In [ ]:
generate_sentinel_measure(data_dict=data_dict, data_dict_practice=data_dict_practice, codelist_dict=codelist_dict, measure='qrisk2', code_column='code', term_column='term', dates_list=["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="cholesterol"></a>
### Serum Cholesterol Level

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'cholesterol', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="ALT"></a>
### ALT Level

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'alt', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="serum_tsh"></a>
### Serum TSH Level

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'tsh', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="rbc_fbc"></a>
### Red Blood Cell Count

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'rbc', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="hba1c"></a>
### Haemoglobin A1c Level - IFCC Standardised

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'hba1c', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)


<a id="serum_sodium"></a>
### Serum Sodium Level

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'sodium', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)

<a id="asthma"></a>
### Asthma Annual Review

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'asthma', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)


<a id="copd"></a>
### Chronic Obstructive Pulmonary Disease Annual Review

In [ ]:
generate_sentinel_measure(data_dict, data_dict_practice, codelist_dict, 'copd', 'code', 'term', ["2020-02-01", "2020-04-01", "2020-12-01"], interactive=False)